In [ ]:
import easyocr
import os
from glob import glob

reader = easyocr.Reader(['en'])

In [84]:
import pytesseract
from tqdm import tqdm
from collections import Counter

THRESH = 0.0

def ocr(fp: str):
    result = reader.readtext(fp)
    if len(result) == 0:
        return None
    conf = result[-1][-1]
    if conf < THRESH:
        return None
    return result[0][1]

def ocr_dir(dir: str):
    image_file_paths = glob(os.path.join(dir, "*.jpg"))
    results = []
    for image_file_path in tqdm(image_file_paths):
        result = ocr(image_file_path)
        if result is not None:
            results.append(result)
    return results, dir

def find_most_common_element_in_arr(arr):
    return Counter(arr).most_common(1)[0][0] if len(arr) > 0 else None

In [85]:
test_img = '/playpen-storage/levlevi/player-re-id/src/testing/ocr_analysis/sample_tracks_nba_100/__tracks_nba_50_grouped__/_hand-labeled/1_1_16/017542.jpg'
test_dir = '/playpen-storage/levlevi/player-re-id/src/testing/ocr_analysis/sample_tracks_nba_100/__tracks_nba_50_grouped__/_hand-labeled/1_1_16'

dirs = glob(os.path.join('/playpen-storage/levlevi/player-re-id/src/testing/ocr_analysis/sample_tracks_nba_100/__tracks_nba_50_grouped__/_hand-labeled', "*"))
folder_paths = []
predictions = []
for dir in dirs:
    results, dir = ocr_dir(dir)
    folder_paths.append(dir)
    predictions.append(find_most_common_element_in_arr(results))

folder_paths
predictions

100%|██████████| 365/365 [00:04<00:00, 91.15it/s]


[None,
 '99',
 None,
 None,
 None,
 None,
 '',
 None,
 None,
 '5',
 '22',
 '5',
 '31',
 None,
 None,
 '15',
 '22',
 '30',
 '5',
 None,
 '2',
 None,
 'EDM)',
 None,
 None,
 '2',
 '16',
 'JU',
 '00',
 None,
 '2',
 'KIn',
 '50',
 None,
 '14',
 None,
 '2',
 '5',
 'Ji',
 None,
 '12',
 None,
 'DV',
 None,
 '33',
 '22',
 None,
 None,
 '21',
 None,
 None,
 '0',
 None,
 '2',
 '12',
 'I2',
 '13',
 None,
 'I',
 '2',
 '15',
 'D',
 None,
 '32',
 '2',
 'A',
 '15',
 '5',
 'K',
 None,
 '30',
 'WOLVES',
 '21',
 None,
 None,
 '10',
 '5',
 '1',
 None,
 '69',
 None,
 '5',
 None,
 'J0',
 'HEF',
 '30',
 None,
 None,
 '32',
 '5',
 'Qud',
 '27',
 None,
 '5',
 None,
 '5',
 '15',
 '5',
 '6',
 None]

In [86]:
predictions[1]

'99'

In [88]:
import pandas as pd

ground_truth_labels = []
for fp in folder_paths:
    label = fp.split('_')[-1]
    ground_truth_labels.append(label)
    
predictions_none_replaced = []
for pred in predictions:
    if pred is None:
        predictions_none_replaced.append('na')
    else:
        predictions_none_replaced.append(pred)

df = pd.DataFrame({'human_annotation': ground_truth_labels, 'prediction': predictions_none_replaced})
len(df[df['human_annotation'] == df['prediction']])

38